In [6]:
import pandas as pd
pd.options.display.max_rows = 10000
pd.set_option('display.max_colwidth', -1)
from collections import Counter

In [11]:
graph='../csv/2015.csv'
network=graph

In [12]:
df=pd.read_csv(network)

In [13]:
dg=df[['id','modularity_class']].groupby('modularity_class').count()
dg=dg.reset_index(drop=True)
dg.rename(columns={'id':'cluster size'},inplace=True)

In [14]:
df['mod_size']=df.modularity_class.apply(lambda x: dg.iloc[x])

In [15]:
#df.sort_values(['mod_size','degree'],ascending=False)[['label','modularity_class']]

In [16]:
docs=df.label

In [17]:
import re
from nltk import stem
porter = stem.porter.PorterStemmer()

docs_cleaned = [[re.sub('[.!,;?()\W\d]+', '', word) for word in doc.lower().split()] for doc in docs]

In [18]:
stop= open ('stopList.txt').read()
stop_list = set(stop.replace("\n", " ").split())
docs_stemed = [[porter.stem(word) for word in document if word not in stop_list] for document in docs_cleaned]

In [19]:
df['ind']=df.index
df['terms']=df.ind.apply(lambda x: docs_stemed[x])

In [21]:
dg['keyword']=''
dg['key terms']=''

for mod in dg.index:
    mod_terms=[]
    for terms in df[df.modularity_class==mod].terms:
        for term in terms:
            mod_terms.append(term)
            
    words = [word for word in mod_terms if word != '']                
    words_counter = Counter(words)
    top_words=[word for word in words_counter.most_common(5) if word[1]>1]
    dg.set_value(mod,'keyword',top_words)
    
    terms = [words[i]+' '+words[i+1] for i in range(len(words)-1)]
    terms_counter= Counter(terms)
    top_terms=[term for term in terms_counter.most_common(5) if term[1]>1]
    dg.set_value(mod,'key terms',top_terms)

In [22]:
density=df.groupby(['modularity_class'])['triangles'].mean()
dg['cluster density']=(density/max(density)).round(2)
dg=dg.sort_values('cluster density',ascending=False).reset_index(drop=True)

In [26]:
dg.to_csv('../csv/clusters_2015.csv', encoding='ascii')
dg

,cluster size,keyword,key terms,cluster density
0,77,"[(knowledg, 62), (manag, 24), (organiz, 13), (process, 11), (system, 10)]","[(knowledg manag, 17), (manag system, 5), (knowledg transfer, 5), (conceptu model, 4), (manag knowledg, 3)]",1.00
1,84,"[(secur, 42), (risk, 28), (system, 19), (technolog, 16), (manag, 14)]","[(real option, 9), (system secur, 7), (secur risk, 6), (risk manag, 6), (technolog invest, 6)]",0.68
2,90,"[(trust, 42), (web, 21), (onlin, 16), (cultur, 14), (site, 10)]","[(web site, 9), (trust onlin, 5), (virtual team, 4), (recommend agent, 3), (technolog accept, 3)]",0.48
3,102,"[(team, 34), (virtual, 32), (collabor, 24), (technolog, 18), (softwar, 13)]","[(virtual world, 11), (virtual team, 9), (softwar develop, 7), (collabor technolog, 5), (project team, 4)]",0.45
4,73,"[(project, 36), (softwar, 19), (system, 19), (control, 17), (offshor, 15)]","[(softwar project, 9), (system develop, 7), (softwar develop, 6), (develop project, 5), (project perform, 5)]",0.43
5,54,"[(auction, 26), (onlin, 20), (search, 19), (market, 9), (advertis, 8)]","[(onlin auction, 8), (combinatori auction, 6), (sponsor search, 4), (search engin, 3), (search market, 3)]",0.36
6,41,"[(privaci, 25), (person, 14), (concern, 9), (model, 8), (onlin, 7)]","[(privaci concern, 7), (web person, 4), (elabor likelihood, 3), (likelihood model, 3), (person onlin, 2)]",0.36
7,137,"[(commun, 44), (group, 42), (social, 28), (media, 28), (support, 23)]","[(support system, 14), (social media, 14), (group support, 13), (onlin commun, 9), (electron brainstorm, 8)]",0.35
8,120,"[(market, 38), (price, 35), (onlin, 23), (electron, 21), (technolog, 15)]","[(electron market, 5), (suppli chain, 5), (empir analysi, 4), (price dispers, 4), (onlin marketplac, 4)]",0.34
9,71,"[(measur, 25), (system, 17), (model, 12), (construct, 9), (form, 8)]","[(servic qualiti, 6), (measur system, 5), (requir elicit, 4), (construct measur, 4), (equat model, 3)]",0.29
